In [1]:
import financeMacroFactors as fM
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta as tDel
from pprint import pprint

# Company Valuation

The library also provides functions that will allow you to get all the company valuations directly without having to remember all the valuation methodologies. Just supply the required data, and this valuation library method will help you calculate the valuations for you.

## First download all fundamental data

In [2]:
fundamentals = fM.companies.getTickerFundamentalDataMW('MSFT')


In [3]:

toDownload = [
    ['eps',     'IncomeStatement', 'EPS (Diluted)' ],
    ['fcf',     'CashFlow',        'Free Cash Flow'],
    ['shares',  'IncomeStatement', 'Diluted Shares Outstanding'],
    ['revenue', 'IncomeStatement', 'Sales/Revenue'],
]

data = {}
dates = None
for key, statement, value in toDownload:
    tempData = fM.companies.extractYearlyData( 
                    fundamentals[statement], value)

    dates, t = zip(*tempData)
    data[key] = np.array(t)

pprint(data)
pprint(dates)

{'eps': array([2.1 , 3.25, 2.13, 5.06, 5.76]),
 'fcf': array([2.498e+10, 3.138e+10, 3.225e+10, 3.826e+10, 4.523e+10]),
 'revenue': array([8.4700e+10, 9.6020e+10, 1.1018e+11, 1.2550e+11, 1.4302e+11]),
 'shares': array([8.01e+09, 7.83e+09, 7.79e+09, 7.75e+09, 7.68e+09])}
(datetime.datetime(2016, 6, 30, 0, 0),
 datetime.datetime(2017, 6, 30, 0, 0),
 datetime.datetime(2018, 6, 30, 0, 0),
 datetime.datetime(2019, 6, 30, 0, 0),
 datetime.datetime(2020, 6, 30, 0, 0))


## Then Download all Pricing Data

This is already described in all the previous examples, so we shall not explain everything here anymore.

In [4]:
stockPrice = fM.companies.yahooData.getStockDataYahoo(
                    'msft', dates[0]-tDel(90), dt.now(), '1mo')

stockPrice = stockPrice[1:]
price = np.array([m[-2] for m in stockPrice if ((m[0]-tDel(1)) in dates)])
price = price[::-1]
print(f'Share Price over the same period = {price}')

Share Price over the same period = [ 52.6   69.15 102.87 134.29 204.52]


In [5]:
dfe  = fM.valuation.discountedFutureEarnings( data['eps']                            )
dcf  = fM.valuation.discountedCashFlow(       data['fcf'],     data['shares']        )
p_s  = fM.valuation.priceToSalesRatio(        data['revenue'], data['shares'], price )
p_e  = fM.valuation.priceToEarningsRatio(     data['eps'],     price                 )

print(f'''
Valuation Method:
    1. Discounted Future Earnings: USD {dfe:6.2f}
    2.       Discounted Cash Flow: USD {dcf:6.2f}
    3.          Price/Sales Ratio: USD {p_s:6.2f}
    4.         Price/Equity Ratio: USD {p_e:6.2f}
''')


Valuation Method:
    1. Discounted Future Earnings: USD  81.04
    2.       Discounted Cash Flow: USD  92.00
    3.          Price/Sales Ratio: USD 138.41
    4.         Price/Equity Ratio: USD 180.48

